# 🔄 Introducción a LangGraph - De Chains Simples a Flujos Inteligentes

## 📌 Objetivo del Notebook

En este notebook aprenderás a usar **LangGraph** para crear agentes con flujos complejos y toma de decisiones.

### ¿Qué construiremos?

1. **Grafo básico**: Primer contacto con nodos y aristas
2. **Grafo condicional**: Flujos que toman decisiones
3. **Agente RAG mejorado**: Aplicando LangGraph a nuestro proyecto real
4. **Sistema de ventas**: Caso de uso completo con validaciones


---
## 📦 Parte 1: Instalación y Setup

### Instalación de paquetes

In [17]:
# Instalar LangGraph y dependencias
!pip install -q langgraph langchain langchain-openai langchain-community
!pip install -q python-dotenv pypdf faiss-cpu

print("✅ Paquetes instalados exitosamente")

✅ Paquetes instalados exitosamente


In [18]:
pip list

Package                                  Version
---------------------------------------- -----------
absl-py                                  2.3.1
aiofiles                                 25.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aiosignal                                1.4.0
alembic                                  1.17.0
annotated-types                          0.7.0
anyio                                    4.11.0
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
asgiref                                  3.9.2
aspose-pdf                               25.8.0
asttokens                                2.0.5
astunparse                               1.6.3
async-lru                                2.0.4
attrs                                    24.3.0
Authlib                                  1.6.5
Babel                                    2.11.0
backoff                                  2.

### Configuración de API Key

In [19]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Verificar API Key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("⚠️ No se encontró OPENAI_API_KEY en el archivo .env")

print("✅ API Key configurada")
print(f"   Longitud: {len(api_key)} caracteres")

✅ API Key configurada
   Longitud: 164 caracteres


---
## 🎯 Parte 2: Tu Primer Grafo con LangGraph

### ¿Qué es un Grafo?

Un grafo es una estructura que conecta **nodos** (pasos) con **aristas** (conexiones).

```
Nodo A ──→ Nodo B ──→ Nodo C ──→ FIN
```

### Componentes:
1. **Estado**: Diccionario compartido entre nodos
2. **Nodos**: Funciones que procesan el estado
3. **Aristas**: Conexiones entre nodos
4. **Grafo**: Estructura que los conecta todos

### Ejemplo 1: Grafo Lineal Simple

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

print("="*70)
print("🎯 EJEMPLO 1: GRAFO LINEAL SIMPLE")
print("="*70)

# PASO 1: Definir el Estado
class EstadoSimple(TypedDict):
    """Estado compartido entre todos los nodos"""
    contador: int 
    mensaje: str 

print("\n1️⃣ Estado definido:")
print("   - contador: int")
print("   - mensaje: str")

# PASO 2: Crear Nodos
def inicializar(state: EstadoSimple) -> EstadoSimple:
    """Nodo 1: Inicializa el contador"""
    print("   🔹 Ejecutando nodo: inicializar")
    state["contador"] = 0
    state["mensaje"] = "Iniciando"
    return state

def incrementar(state: EstadoSimple) -> EstadoSimple:
    """Nodo 2: Incrementa el contador"""
    print("   🔹 Ejecutando nodo: incrementar")
    state["contador"] += 1
    state["mensaje"] = "Incrementado"
    return state

def finalizar(state: EstadoSimple) -> EstadoSimple:
    """Nodo 3: Finaliza"""
    print("   🔹 Ejecutando nodo: finalizar")
    state["mensaje"] = "Completado"
    return state

print("\n2️⃣ Nodos creados:")
print("   - inicializar")
print("   - incrementar")
print("   - finalizar")

# PASO 3: Crear el Grafo
graph = StateGraph(EstadoSimple)

# PASO 4: Agregar Nodos al Grafo
graph.add_node("inicializar", inicializar)
graph.add_node("incrementar", incrementar)
graph.add_node("finalizar", finalizar)

print("\n3️⃣ Nodos agregados al grafo")

# PASO 5: Conectar Nodos con Aristas
graph.set_entry_point("inicializar")  # Empieza aquí
graph.add_edge("inicializar", "incrementar")  # inicializar → incrementar
graph.add_edge("incrementar", "finalizar")    # incrementar → finalizar
graph.add_edge("finalizar", END)              # finalizar → FIN

print("\n4️⃣ Aristas creadas:")
print("   inicializar → incrementar → finalizar → END")

# PASO 6: Compilar el Grafo
app = graph.compile()

print("\n5️⃣ Grafo compilado exitosamente")

# PASO 7: Ejecutar
print("\n" + "="*70)
print("🚀 EJECUTANDO EL GRAFO")
print("="*70)

estado_inicial = {"contador": 0, "mensaje": ""}
print(f"\n📥 Estado inicial: {estado_inicial}")

print("\n⚙️ Ejecutando nodos:")
resultado = app.invoke(estado_inicial)

print("\n" + "="*70)
print("✅ RESULTADO FINAL")
print("="*70)
print(f"📤 Estado final: {resultado}")
print(f"\n   Contador: {resultado['contador']}")
print(f"   Mensaje: {resultado['mensaje']}")

🎯 EJEMPLO 1: GRAFO LINEAL SIMPLE

1️⃣ Estado definido:
   - contador: int
   - mensaje: str

2️⃣ Nodos creados:
   - inicializar
   - incrementar
   - finalizar

3️⃣ Nodos agregados al grafo

4️⃣ Aristas creadas:
   inicializar → incrementar → finalizar → END

5️⃣ Grafo compilado exitosamente

🚀 EJECUTANDO EL GRAFO

📥 Estado inicial: {'contador': 0, 'mensaje': ''}

⚙️ Ejecutando nodos:
   🔹 Ejecutando nodo: inicializar
   🔹 Ejecutando nodo: incrementar
   🔹 Ejecutando nodo: finalizar

✅ RESULTADO FINAL
📤 Estado final: {'contador': 1, 'mensaje': 'Completado'}

   Contador: 1
   Mensaje: Completado


### 💡 Observaciones Importantes

**¿Qué pasó?**
1. Cada nodo modificó el estado compartido
2. Los nodos se ejecutaron en orden lineal
3. El estado se pasó automáticamente entre nodos

**¿Por qué es útil?**
- No necesitas pasar parámetros manualmente
- El estado es accesible para todos los nodos
- Fácil de visualizar y debuggear

---
## 🔀 Parte 3: Grafo con Decisiones Condicionales

### ¿Qué son las Aristas Condicionales?

Permiten que el grafo **decida** a qué nodo ir según el estado.

```
       Nodo A
          │
    ¿Condición?
     /       \
   Sí         No
    │          │
Nodo B      Nodo C
```

### Ejemplo 2: Clasificador de Números

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END

print("="*70)
print("🎯 EJEMPLO 2: GRAFO CON DECISIONES")
print("="*70)

# Estado
class EstadoNumero(TypedDict):
    numero: int
    tipo: str
    resultado: str

# Nodos
def recibir_numero(state: EstadoNumero) -> EstadoNumero:
    """Nodo inicial que recibe el número"""
    print(f"\n📥 Número recibido: {state['numero']}")
    return state

def es_par(state: EstadoNumero) -> EstadoNumero:
    """Procesa números pares"""
    print("   ✅ Es un número PAR")
    state["tipo"] = "par"
    state["resultado"] = f"El número {state['numero']} es PAR"
    return state

def es_impar(state: EstadoNumero) -> EstadoNumero:
    """Procesa números impares"""
    print("   ✅ Es un número IMPAR")
    state["tipo"] = "impar"
    state["resultado"] = f"El número {state['numero']} es IMPAR"
    return state

# Función de decisión
def decidir_tipo(state: EstadoNumero) -> Literal["par", "impar"]:
    """Decide a qué nodo ir según si el número es par o impar"""
    if state["numero"] % 2 == 0:
        print("   🔀 Decisión: Ir a nodo 'par'")
        return "par"
    else:
        print("   🔀 Decisión: Ir a nodo 'impar'")
        return "impar"

# Crear grafo
graph = StateGraph(EstadoNumero)

# Agregar nodos
graph.add_node("recibir", recibir_numero)
graph.add_node("par", es_par)
graph.add_node("impar", es_impar)

# Punto de entrada
graph.set_entry_point("recibir")

# Arista condicional: decide según el resultado de decidir_tipo()
graph.add_conditional_edges(
    "recibir",          # Desde este nodo
    decidir_tipo,       # Función que decide
    {
        "par": "par",     # Si retorna "par", ir a nodo "par"
        "impar": "impar" # Si retorna "impar", ir a nodo "impar"
    }
)

# Ambos nodos terminan
graph.add_edge("par", END)
graph.add_edge("impar", END)

# Compilar
app = graph.compile()

print("\n" + "="*70)
print("🧪 PRUEBAS")
print("="*70)

# Probar con número par
print("\n[Prueba 1: Número PAR]")
resultado1 = app.invoke({"numero": 8, "tipo": "", "resultado": ""})
print(f"📤 Resultado: {resultado1['resultado']}")

# Probar con número impar
print("\n[Prueba 2: Número IMPAR]")
resultado2 = app.invoke({"numero": 7, "tipo": "", "resultado": ""})
print(f"📤 Resultado: {resultado2['resultado']}")

🎯 EJEMPLO 2: GRAFO CON DECISIONES

🧪 PRUEBAS

[Prueba 1: Número PAR]

📥 Número recibido: 8
   🔀 Decisión: Ir a nodo 'par'
   ✅ Es un número PAR
📤 Resultado: El número 8 es PAR

[Prueba 2: Número IMPAR]

📥 Número recibido: 7
   🔀 Decisión: Ir a nodo 'impar'
   ✅ Es un número IMPAR
📤 Resultado: El número 7 es IMPAR


### 💡 ¿Qué aprendimos?

**Aristas Condicionales:**
- La función `decidir_tipo()` retorna una string que determina el siguiente nodo
- Esto permite crear flujos dinámicos basados en el estado
- Es la base para crear agentes inteligentes

---
## 🏗️ Parte 4: Caso Práctico - Sistema de Validación de Cliente

### Escenario Real:

Un cliente quiere rentar un equipo, pero primero debemos:
1. Validar que sea cliente registrado
2. Si no es cliente, registrarlo
3. Clasificar su consulta (producto, precio, disponibilidad)
4. Responder según el tipo de consulta

In [ ]:
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, END

print("="*70)
print("🎯 EJEMPLO 3: SISTEMA DE VALIDACIÓN DE CLIENTE")
print("="*70)

# Estado
class EstadoCliente(TypedDict):
    cliente_id: str
    nombre: str
    es_cliente: bool
    consulta: str
    tipo_consulta: str  # "producto", "precio", "disponibilidad"
    respuesta: str

# Base de datos simulada
CLIENTES_DB = {
    "C001": "Juan Pérez",
    "C002": "María González",
}

# Nodos
def validar_cliente(state: EstadoCliente) -> EstadoCliente:
    """Valida si el cliente está registrado"""
    print(f"\n🔍 Validando cliente ID: {state['cliente_id']}")
    
    if state["cliente_id"] in CLIENTES_DB:
        state["es_cliente"] = True
        state["nombre"] = CLIENTES_DB[state["cliente_id"]]
        print(f"   ✅ Cliente encontrado: {state['nombre']}")
    else:
        state["es_cliente"] = False
        print(f"   ❌ Cliente no encontrado")
    
    return state

def registrar_cliente(state: EstadoCliente) -> EstadoCliente:
    """Registra un nuevo cliente"""
    print(f"\n📝 Registrando nuevo cliente...")
    CLIENTES_DB[state["cliente_id"]] = state["nombre"]
    state["es_cliente"] = True
    state["respuesta"] = f"Bienvenido {state['nombre']}! Has sido registrado exitosamente."
    print(f"   ✅ Cliente registrado: {state['nombre']}")
    return state

def clasificar_consulta(state: EstadoCliente) -> EstadoCliente:
    """Clasifica el tipo de consulta"""
    print(f"\n🔍 Clasificando consulta: '{state['consulta']}'")
    
    consulta_lower = state["consulta"].lower()
    
    if any(word in consulta_lower for word in ["precio", "costo", "cuánto", "cuanto"]):
        state["tipo_consulta"] = "precio"
        print("   📊 Tipo: PRECIO")
    elif any(word in consulta_lower for word in ["disponible", "hay", "tienen", "stock"]):
        state["tipo_consulta"] = "disponibilidad"
        print("   📦 Tipo: DISPONIBILIDAD")
    else:
        state["tipo_consulta"] = "producto"
        print("   🛠️ Tipo: PRODUCTO")
    
    return state

def responder_precio(state: EstadoCliente) -> EstadoCliente:
    """Responde consultas sobre precio"""
    print(f"\n💰 Procesando consulta de precio...")
    state["respuesta"] = f"Hola {state['nombre']}, nuestros equipos tienen precios desde L500/día. ¿Qué equipo específico te interesa?"
    return state

def responder_disponibilidad(state: EstadoCliente) -> EstadoCliente:
    """Responde consultas sobre disponibilidad"""
    print(f"\n📦 Procesando consulta de disponibilidad...")
    state["respuesta"] = f"Hola {state['nombre']}, tenemos disponibilidad de rotomartillos, demoledores y compactadores."
    return state

def responder_producto(state: EstadoCliente) -> EstadoCliente:
    """Responde consultas generales sobre productos"""
    print(f"\n🛠️ Procesando consulta general...")
    state["respuesta"] = f"Hola {state['nombre']}, ofrecemos equipos de construcción de alta calidad. ¿En qué podemos ayudarte?"
    return state

# Funciones de decisión
def decidir_registro(state: EstadoCliente) -> Literal["registrar", "clasificar"]:
    """Decide si hay que registrar al cliente"""
    if state["es_cliente"]:
        print("   🔀 Decisión: Cliente ya registrado → clasificar")
        return "clasificar"
    else:
        print("   🔀 Decisión: Cliente nuevo → registrar")
        return "registrar"

def decidir_respuesta(state: EstadoCliente) -> Literal["precio", "disponibilidad", "producto"]:
    """Decide qué tipo de respuesta dar"""
    print(f"   🔀 Decisión: Responder con tipo '{state['tipo_consulta']}'")
    return state["tipo_consulta"]

# Crear grafo
graph = StateGraph(EstadoCliente)

# Agregar nodos
graph.add_node("validar", validar_cliente)
graph.add_node("registrar", registrar_cliente)
graph.add_node("clasificar", clasificar_consulta)
graph.add_node("responder_precio", responder_precio)
graph.add_node("responder_disponibilidad", responder_disponibilidad)
graph.add_node("responder_producto", responder_producto)

# Conectar
graph.set_entry_point("validar")

# Si no es cliente → registrar, si es cliente → clasificar
graph.add_conditional_edges(
    "validar",
    decidir_registro,
    {
        "registrar": "registrar",
        "clasificar": "clasificar"
    }
)

# Después de registrar → END (ya dio respuesta de bienvenida)
graph.add_edge("registrar", END)

# Después de clasificar → decidir tipo de respuesta
graph.add_conditional_edges(
    "clasificar",
    decidir_respuesta,
    {
        "precio": "responder_precio",
        "disponibilidad": "responder_disponibilidad",
        "producto": "responder_producto"
    }
)

# Todas las respuestas terminan
graph.add_edge("responder_precio", END)
graph.add_edge("responder_disponibilidad", END)
graph.add_edge("responder_producto", END)

# Compilar
app = graph.compile()

print("\n" + "="*70)
print("🧪 PRUEBAS DEL SISTEMA")
print("="*70)

🎯 EJEMPLO 3: SISTEMA DE VALIDACIÓN DE CLIENTE

🧪 PRUEBAS DEL SISTEMA


### Prueba 1: Cliente existente pregunta sobre precio

In [7]:
print("\n" + "="*70)
print("[PRUEBA 1: Cliente existente - Consulta de PRECIO]")
print("="*70)

resultado1 = app.invoke({
    "cliente_id": "C001",
    "nombre": "",
    "es_cliente": False,
    "consulta": "¿Cuánto cuesta rentar un rotomartillo?",
    "tipo_consulta": "",
    "respuesta": ""
})

print("\n" + "="*70)
print("✅ RESPUESTA FINAL")
print("="*70)
print(f"Cliente: {resultado1['nombre']}")
print(f"Tipo de consulta: {resultado1['tipo_consulta']}")
print(f"\nRespuesta:\n{resultado1['respuesta']}")


[PRUEBA 1: Cliente existente - Consulta de PRECIO]

🔍 Validando cliente ID: C001
   ✅ Cliente encontrado: Juan Pérez
   🔀 Decisión: Cliente ya registrado → clasificar

🔍 Clasificando consulta: '¿Cuánto cuesta rentar un rotomartillo?'
   📊 Tipo: PRECIO
   🔀 Decisión: Responder con tipo 'precio'

💰 Procesando consulta de precio...

✅ RESPUESTA FINAL
Cliente: Juan Pérez
Tipo de consulta: precio

Respuesta:
Hola Juan Pérez, nuestros equipos tienen precios desde L500/día. ¿Qué equipo específico te interesa?


### Prueba 2: Cliente nuevo quiere información

In [8]:
print("\n" + "="*70)
print("[PRUEBA 2: Cliente NUEVO]")
print("="*70)

resultado2 = app.invoke({
    "cliente_id": "C003",
    "nombre": "Pedro Martínez",
    "es_cliente": False,
    "consulta": "Necesito información sobre equipos",
    "tipo_consulta": "",
    "respuesta": ""
})

print("\n" + "="*70)
print("✅ RESPUESTA FINAL")
print("="*70)
print(f"Cliente: {resultado2['nombre']}")
print(f"\nRespuesta:\n{resultado2['respuesta']}")

print("\n📊 Base de datos actualizada:")
print(f"   Clientes registrados: {len(CLIENTES_DB)}")
for id, nombre in CLIENTES_DB.items():
    print(f"   - {id}: {nombre}")


[PRUEBA 2: Cliente NUEVO]

🔍 Validando cliente ID: C003
   ❌ Cliente no encontrado
   🔀 Decisión: Cliente nuevo → registrar

📝 Registrando nuevo cliente...
   ✅ Cliente registrado: Pedro Martínez

✅ RESPUESTA FINAL
Cliente: Pedro Martínez

Respuesta:
Bienvenido Pedro Martínez! Has sido registrado exitosamente.

📊 Base de datos actualizada:
   Clientes registrados: 3
   - C001: Juan Pérez
   - C002: María González
   - C003: Pedro Martínez


### Prueba 3: Consulta sobre disponibilidad

In [9]:
print("\n" + "="*70)
print("[PRUEBA 3: Cliente existente - Consulta de DISPONIBILIDAD]")
print("="*70)

resultado3 = app.invoke({
    "cliente_id": "C002",
    "nombre": "",
    "es_cliente": False,
    "consulta": "¿Tienen demoledores disponibles?",
    "tipo_consulta": "",
    "respuesta": ""
})

print("\n" + "="*70)
print("✅ RESPUESTA FINAL")
print("="*70)
print(f"Cliente: {resultado3['nombre']}")
print(f"Tipo de consulta: {resultado3['tipo_consulta']}")
print(f"\nRespuesta:\n{resultado3['respuesta']}")


[PRUEBA 3: Cliente existente - Consulta de DISPONIBILIDAD]

🔍 Validando cliente ID: C002
   ✅ Cliente encontrado: María González
   🔀 Decisión: Cliente ya registrado → clasificar

🔍 Clasificando consulta: '¿Tienen demoledores disponibles?'
   📦 Tipo: DISPONIBILIDAD
   🔀 Decisión: Responder con tipo 'disponibilidad'

📦 Procesando consulta de disponibilidad...

✅ RESPUESTA FINAL
Cliente: María González
Tipo de consulta: disponibilidad

Respuesta:
Hola María González, tenemos disponibilidad de rotomartillos, demoledores y compactadores.


### 💡 ¿Qué logramos?

**Este grafo es MUCHO más inteligente que una chain simple porque:**

1. ✅ **Valida clientes** automáticamente
2. ✅ **Registra nuevos clientes** en el flujo
3. ✅ **Clasifica consultas** inteligentemente
4. ✅ **Responde diferente** según el tipo de consulta
5. ✅ **Toma decisiones** en múltiples puntos

**vs una chain simple que solo haría:**
```python
respuesta = llm.invoke(consulta)  # Sin validaciones ni lógica
```

---
## 🤖 Parte 5: Integrando LLM con LangGraph

### Ahora vamos a crear un grafo que use el LLM de OpenAI

**Caso de uso:** Sistema que decide si necesita buscar en documentos o responder directamente

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

print("="*70)
print("🎯 EJEMPLO 4: GRAFO CON LLM")
print("="*70)

# Inicializar LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
print("\n✅ LLM inicializado (gpt-4o-mini)")

# Estado
class EstadoLLM(TypedDict):
    pregunta: str
    es_pregunta_tecnica: bool
    necesita_documentos: bool
    respuesta: str

# Nodos
def analizar_pregunta(state: EstadoLLM) -> EstadoLLM:
    """Usa el LLM para analizar si la pregunta es técnica"""
    print(f"\n🔍 Analizando pregunta: '{state['pregunta']}'")
    
    prompt = f"""Analiza esta pregunta y responde SOLO 'SI' o 'NO'.
    
    ¿La pregunta requiere información técnica específica sobre productos de construcción?
    
    Pregunta: {state['pregunta']}
    
    Respuesta:"""
    
    respuesta = llm.invoke([HumanMessage(content=prompt)])
    
    es_tecnica = "si" in respuesta.content.lower()
    state["es_pregunta_tecnica"] = es_tecnica
    
    if es_tecnica:
        print("   ✅ Es pregunta TÉCNICA")
    else:
        print("   ✅ Es pregunta GENERAL")
    
    return state

def buscar_en_documentos(state: EstadoLLM) -> EstadoLLM:
    """Simula búsqueda en documentos"""
    print("\n📚 Buscando en documentos técnicos...")
    # Aquí iría la búsqueda real en FAISS
    contexto = """ROTOMARTILLO TE-70
    Precio: L750 por día
    Potencia: 1000W
    Uso: Perforación en concreto
    Disponibilidad: En stock"""
    
    print("   ✅ Documentos encontrados")
    
    prompt = f"""Basándote en esta información técnica, responde la pregunta del usuario.
    
    INFORMACIÓN TÉCNICA:
    {contexto}
    
    PREGUNTA:
    {state['pregunta']}
    
    RESPUESTA (en español, profesional):"""
    
    respuesta = llm.invoke([HumanMessage(content=prompt)])
    state["respuesta"] = respuesta.content
    
    return state

def responder_directamente(state: EstadoLLM) -> EstadoLLM:
    """Responde sin buscar documentos"""
    print("\n💬 Respondiendo directamente...")
    
    prompt = f"""Eres un asistente amable de una empresa de renta de equipos.
    Responde esta pregunta de forma breve y profesional.
    
    Pregunta: {state['pregunta']}
    
    Respuesta:"""
    
    respuesta = llm.invoke([HumanMessage(content=prompt)])
    state["respuesta"] = respuesta.content
    
    return state

# Decisión
def decidir_flujo(state: EstadoLLM) -> Literal["buscar", "directa"]:
    """Decide si buscar en docs o responder directo"""
    if state["es_pregunta_tecnica"]:
        print("   🔀 Decisión: Buscar en documentos")
        return "buscar"
    else:
        print("   🔀 Decisión: Responder directamente")
        return "directa"

# Crear grafo
graph = StateGraph(EstadoLLM)

graph.add_node("analizar", analizar_pregunta)
graph.add_node("buscar", buscar_en_documentos)
graph.add_node("directa", responder_directamente)

graph.set_entry_point("analizar")

graph.add_conditional_edges(
    "analizar",
    decidir_flujo,
    {
        "buscar": "buscar",
        "directa": "directa"
    }
)

graph.add_edge("buscar", END)
graph.add_edge("directa", END)

app = graph.compile()

print("\n✅ Grafo compilado")

🎯 EJEMPLO 4: GRAFO CON LLM

✅ LLM inicializado (gpt-4o-mini)

✅ Grafo compilado


### Probar con diferentes preguntas

In [11]:
# Pregunta técnica
print("\n" + "="*70)
print("[PRUEBA 1: Pregunta TÉCNICA]")
print("="*70)

resultado1 = app.invoke({
    "pregunta": "¿Cuál es la potencia del rotomartillo TE-70?",
    "es_pregunta_tecnica": False,
    "necesita_documentos": False,
    "respuesta": ""
})

print("\n" + "="*70)
print("📤 RESPUESTA")
print("="*70)
print(resultado1["respuesta"])


[PRUEBA 1: Pregunta TÉCNICA]

🔍 Analizando pregunta: '¿Cuál es la potencia del rotomartillo TE-70?'
   ✅ Es pregunta TÉCNICA
   🔀 Decisión: Buscar en documentos

📚 Buscando en documentos técnicos...
   ✅ Documentos encontrados

📤 RESPUESTA
La potencia del rotomartillo TE-70 es de 1000W.


In [12]:
# Pregunta general
print("\n" + "="*70)
print("[PRUEBA 2: Pregunta GENERAL]")
print("="*70)

resultado2 = app.invoke({
    "pregunta": "¿A qué hora abren?",
    "es_pregunta_tecnica": False,
    "necesita_documentos": False,
    "respuesta": ""
})

print("\n" + "="*70)
print("📤 RESPUESTA")
print("="*70)
print(resultado2["respuesta"])


[PRUEBA 2: Pregunta GENERAL]

🔍 Analizando pregunta: '¿A qué hora abren?'
   ✅ Es pregunta GENERAL
   🔀 Decisión: Responder directamente

💬 Respondiendo directamente...

📤 RESPUESTA
Nuestra empresa abre a las 8:00 a.m. y cierra a las 6:00 p.m. de lunes a viernes. Si necesita más información, no dude en preguntar.


### 💡 Observaciones

**¿Qué pasó?**
1. El grafo analizó si la pregunta era técnica
2. Tomó una decisión automática
3. Ejecutó el flujo correspondiente

**Ventaja vs Chain simple:**
- No hace búsquedas innecesarias para preguntas simples
- Ahorra tokens (costo) y tiempo
- Respuestas más apropiadas según el tipo de pregunta

---
## 🔁 Parte 6: Grafos con Ciclos (Loops)

### ¿Qué son los ciclos?

Permiten que un nodo **vuelva a ejecutarse** hasta cumplir una condición.

```
Inicio → Intentar → ¿Éxito?
                    /      \
                  No       Sí
                   │        │
         ←─────────┘       Fin
         (volver a intentar)
```

### Ejemplo 5: Sistema de Búsqueda con Reintentos

In [13]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END

print("="*70)
print("🎯 EJEMPLO 5: GRAFO CON CICLOS (REINTENTOS)")
print("="*70)

# Estado
class EstadoBusqueda(TypedDict):
    query: str
    intentos: int
    max_intentos: int
    resultados_encontrados: bool
    resultados: str

# Base de datos simulada
PRODUCTOS = {
    "rotomartillo": "Rotomartillo TE-70, L750/día",
    "demoledor": "Demoledor TE-2000, L1100/día",
    "compactador": "Compactador 524 KGS, L900/día"
}

# Nodos
def buscar(state: EstadoBusqueda) -> EstadoBusqueda:
    """Intenta buscar resultados"""
    state["intentos"] += 1
    print(f"\n🔍 Intento {state['intentos']}/{state['max_intentos']}")
    print(f"   Query: '{state['query']}'")
    
    # Buscar en "base de datos"
    query_lower = state["query"].lower()
    
    encontrado = False
    for key, value in PRODUCTOS.items():
        if key in query_lower:
            state["resultados"] = value
            state["resultados_encontrados"] = True
            encontrado = True
            print(f"   ✅ Encontrado: {value}")
            break
    
    if not encontrado:
        state["resultados_encontrados"] = False
        print("   ❌ No se encontraron resultados")
    
    return state

def reformular_query(state: EstadoBusqueda) -> EstadoBusqueda:
    """Reformula la query para reintentar"""
    print("\n🔄 Reformulando query...")
    
    # Simplificar: quitar palabras comunes
    palabras_eliminar = ["el", "la", "un", "una", "de", "necesito", "quiero"]
    
    palabras = state["query"].lower().split()
    palabras_filtradas = [p for p in palabras if p not in palabras_eliminar]
    
    state["query"] = " ".join(palabras_filtradas)
    print(f"   Nueva query: '{state['query']}'")
    
    return state

def sin_resultados(state: EstadoBusqueda) -> EstadoBusqueda:
    """Maneja el caso sin resultados"""
    print("\n❌ No se encontraron resultados después de todos los intentos")
    state["resultados"] = "Lo siento, no encontré ese producto en nuestro catálogo."
    return state

# Decisión
def decidir_siguiente_paso(state: EstadoBusqueda) -> Literal["fin", "reformular", "sin_resultados"]:
    """Decide si continuar buscando, reformular o terminar"""
    
    # Si encontró resultados → terminar
    if state["resultados_encontrados"]:
        print("   🔀 Decisión: Resultados encontrados → FIN")
        return "fin"
    
    # Si no encontró y aún tiene intentos → reformular
    if state["intentos"] < state["max_intentos"]:
        print(f"   🔀 Decisión: Reintentar (intentos: {state['intentos']}/{state['max_intentos']})")
        return "reformular"
    
    # Si agotó intentos → sin resultados
    print("   🔀 Decisión: Intentos agotados → Sin resultados")
    return "sin_resultados"

# Crear grafo
graph = StateGraph(EstadoBusqueda)

graph.add_node("buscar", buscar)
graph.add_node("reformular", reformular_query)
graph.add_node("sin_resultados", sin_resultados)

graph.set_entry_point("buscar")

# Después de buscar, decidir
graph.add_conditional_edges(
    "buscar",
    decidir_siguiente_paso,
    {
        "fin": END,
        "reformular": "reformular",
        "sin_resultados": "sin_resultados"
    }
)

# Después de reformular → volver a buscar (CICLO)
graph.add_edge("reformular", "buscar")

# Sin resultados termina
graph.add_edge("sin_resultados", END)

app = graph.compile()

print("\n✅ Grafo con ciclo compilado")

🎯 EJEMPLO 5: GRAFO CON CICLOS (REINTENTOS)

✅ Grafo con ciclo compilado


### Prueba 1: Query que necesita reformulación

In [14]:
print("\n" + "="*70)
print("[PRUEBA 1: Query compleja que necesita reformulación]")
print("="*70)

resultado1 = app.invoke({
    "query": "necesito un rotomartillo para el trabajo",
    "intentos": 0,
    "max_intentos": 3,
    "resultados_encontrados": False,
    "resultados": ""
})

print("\n" + "="*70)
print("📤 RESULTADO FINAL")
print("="*70)
print(f"Intentos realizados: {resultado1['intentos']}")
print(f"Resultados: {resultado1['resultados']}")


[PRUEBA 1: Query compleja que necesita reformulación]

🔍 Intento 1/3
   Query: 'necesito un rotomartillo para el trabajo'
   ✅ Encontrado: Rotomartillo TE-70, L750/día
   🔀 Decisión: Resultados encontrados → FIN

📤 RESULTADO FINAL
Intentos realizados: 1
Resultados: Rotomartillo TE-70, L750/día


### Prueba 2: Query que no encuentra nada

In [15]:
print("\n" + "="*70)
print("[PRUEBA 2: Query que no existe]")
print("="*70)

resultado2 = app.invoke({
    "query": "quiero una excavadora",
    "intentos": 0,
    "max_intentos": 3,
    "resultados_encontrados": False,
    "resultados": ""
})

print("\n" + "="*70)
print("📤 RESULTADO FINAL")
print("="*70)
print(f"Intentos realizados: {resultado2['intentos']}")
print(f"Resultados: {resultado2['resultados']}")


[PRUEBA 2: Query que no existe]

🔍 Intento 1/3
   Query: 'quiero una excavadora'
   ❌ No se encontraron resultados
   🔀 Decisión: Reintentar (intentos: 1/3)

🔄 Reformulando query...
   Nueva query: 'excavadora'

🔍 Intento 2/3
   Query: 'excavadora'
   ❌ No se encontraron resultados
   🔀 Decisión: Reintentar (intentos: 2/3)

🔄 Reformulando query...
   Nueva query: 'excavadora'

🔍 Intento 3/3
   Query: 'excavadora'
   ❌ No se encontraron resultados
   🔀 Decisión: Intentos agotados → Sin resultados

❌ No se encontraron resultados después de todos los intentos

📤 RESULTADO FINAL
Intentos realizados: 3
Resultados: Lo siento, no encontré ese producto en nuestro catálogo.


### 💡 ¿Qué aprendimos?

**Los ciclos permiten:**
1. ✅ Reintentar operaciones fallidas
2. ✅ Reformular queries automáticamente
3. ✅ Limitar intentos para evitar loops infinitos
4. ✅ Manejo robusto de casos donde no se encuentra información

**Esto es imposible con una chain simple** que solo ejecuta una vez.

---
## 📊 Parte 7: Comparación Final

### Chain Simple vs LangGraph

In [16]:
print("="*70)
print("📊 COMPARACIÓN: CHAIN vs LANGGRAPH")
print("="*70)

comparacion = """
┌────────────────────────────┬──────────────────┬──────────────────┐
│ Característica             │ Chain Simple     │ LangGraph        │
├────────────────────────────┼──────────────────┼──────────────────┤
│ Flujo                      │ Lineal           │ Condicional      │
│ Decisiones                 │ No               │ Sí               │
│ Reintentos                 │ No               │ Sí (ciclos)      │
│ Validaciones               │ Limitadas        │ Múltiples pasos  │
│ Estado compartido          │ Básico           │ Completo         │
│ Manejo de errores          │ Básico           │ Robusto          │
│ Complejidad                │ Baja             │ Media-Alta       │
│ Flexibilidad               │ Baja             │ Muy Alta         │
│ Visualización              │ Difícil          │ Fácil (grafo)    │
│ Debugging                  │ Complicado       │ Paso a paso      │
│ Casos de uso               │ Simples          │ Complejos        │
│ Producción enterprise      │ Limitado         │ Ideal            │
└────────────────────────────┴──────────────────┴──────────────────┘

Ejemplo de flujo:

CHAIN SIMPLE:
Input → LLM → Output

LANGGRAPH:
Input → Validar → ¿Cliente? → No → Registrar → Continuar
                             → Sí → Continuar
        → Clasificar → ¿Tipo?
                     → Técnica → Buscar → ¿Encontró? → No → Reformular → Buscar
                                                      → Sí → Responder
                     → General → Responder directo
"""

print(comparacion)

📊 COMPARACIÓN: CHAIN vs LANGGRAPH

┌────────────────────────────┬──────────────────┬──────────────────┐
│ Característica             │ Chain Simple     │ LangGraph        │
├────────────────────────────┼──────────────────┼──────────────────┤
│ Flujo                      │ Lineal           │ Condicional      │
│ Decisiones                 │ No               │ Sí               │
│ Reintentos                 │ No               │ Sí (ciclos)      │
│ Validaciones               │ Limitadas        │ Múltiples pasos  │
│ Estado compartido          │ Básico           │ Completo         │
│ Manejo de errores          │ Básico           │ Robusto          │
│ Complejidad                │ Baja             │ Media-Alta       │
│ Flexibilidad               │ Baja             │ Muy Alta         │
│ Visualización              │ Difícil          │ Fácil (grafo)    │
│ Debugging                  │ Complicado       │ Paso a paso      │
│ Casos de uso               │ Simples          │ Complejos        │

---
## 🎓 Resumen del Notebook

### ✅ Lo que aprendiste:

1. **Grafos básicos**: Nodos, aristas, estado compartido
2. **Aristas condicionales**: Tomar decisiones según el estado
3. **Integración con LLM**: Usar OpenAI dentro del grafo
4. **Ciclos**: Reintentar operaciones hasta cumplir condición
5. **Validaciones**: Múltiples puntos de decisión

### 🚀 Próximos Pasos:

1. Revisar el notebook `RAG_OpenAI_V5.ipynb`
2. Ver cómo aplicar LangGraph al proyecto real
3. Integrar con FastAPI en `api_rag_v5.py`
4. Crear tus propios grafos personalizados

### 💡 Puntos Clave:

- **LangGraph NO reemplaza Chains**, sino que las complementa
- **Usa Chains** para flujos lineales simples
- **Usa LangGraph** cuando necesites decisiones, validaciones, reintentos
- **El estado compartido** es el corazón de LangGraph
- **Las aristas condicionales** dan el poder de tomar decisiones

---

## 📚 Recursos Adicionales

- [Documentación de LangGraph](https://langchain-ai.github.io/langgraph/)
- [Ejemplos oficiales](https://github.com/langchain-ai/langgraph/tree/main/examples)
- [Tutorial completo](https://langchain-ai.github.io/langgraph/tutorials/introduction/)

---

**¡Felicitaciones! Ahora sabes crear flujos inteligentes con LangGraph 🎉**